In [ ]:
# import libraries
import tensorflow as tf # for modell training
import matplotlib.pyplot as plt # to show graphical results
from matplotlib.ticker import MaxNLocator

In [ ]:
# own modules
import modules.config as config
from modules.ai_dataset import ai_dataset
from modules.export_trained_model import export_trained_model

In [ ]:
# import training and testing data
test_data = ai_dataset(config.TEST_DATA, config.IMG_SIZE, True)
training_data = ai_dataset(config.TRAINING_DATA, config.IMG_SIZE, True)

In [ ]:
training_data.get_status()

In [ ]:
test_data.get_status()

In [ ]:
# show an image from the training data
plt.figure()
plt.imshow(training_data.get_image(0))
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# verify preperation
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(training_data.get_image(i))
    plt.xlabel(training_data.get_label(i), color="blue")
plt.show()

In [ ]:
# validate what hardware tf.keras is using for computing
tf.config.list_physical_devices()

In [ ]:
# Create the convolutional base
model = tf.keras.models.Sequential(name="tf_model")
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(config.IMG_SIZE, config.IMG_SIZE, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

In [ ]:
# Add Dense layers on top
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(training_data.get_labels_count()))

model.summary()

In [ ]:
training_data.get_tf_images().get_shape()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# train the model
history = model.fit(training_data.get_tf_images(), training_data.get_tf_labels(), epochs=config.TRAINING_EPOCHS, 
                    validation_data=(test_data.get_tf_images(), test_data.get_tf_labels()))

In [ ]:
# Plot learning curve
fig = plt.figure()
ax1 = fig.add_subplot()
ax2 = ax1.twinx()
ax1.plot(history.history["accuracy"], label = "accuracy", color = "k")
ax1.plot(history.history["val_accuracy"], label = "val_accuracy", color = "g")
ax2.plot(history.history["loss"], label = "loss", color = "r")
ax2.plot(history.history["val_loss"], label = "val_loss", color = "y")
ax1.set_xlabel("Epoch")
ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
ax1.set_ylabel("Accuracy")
ax2.set_ylabel("Loss")
fig.legend(loc="upper center", bbox_to_anchor=(0.5, 1., 0., 0.), ncol=4)

In [ ]:
# evaluate model
test_loss, test_acc = model.evaluate(test_data.get_tf_images(), test_data.get_tf_labels(), verbose=2)

In [ ]:
model

In [ ]:
export_trained_model(model, config.TRAINING_EPOCHS, test_loss, test_acc, history, fig, training_data, config.MODEL_PATH)